In [ ]:
from arcgis import GIS
from arcgis.features import FeatureLayer
from arcgis.mapping import WebMap
import tempfile
import json, os
from bs4 import BeautifulSoup

In [ ]:
user_name = input('Username: ')

arcgis_url = 'https://www.arcgis.com'
gis = GIS(arcgis_url, user_name)

print ('succesfully logged in')

In [ ]:
target = gis

In [ ]:
folder_name = 'Copy ODs'

In [ ]:
TEXT_BASED_ITEM_TYPES = frozenset(['Web Map', 'Feature Service', 'Map Service','Web Scene',
                                   'Image Service', 'Feature Collection',
                                   'Feature Collection Template',
                                   'Web Mapping Application', 'Mobile Application',
                                   'Symbol Set', 'Color Set',
                                   'Windows Viewer Configuration', 'Dashboard'])

FILE_BASED_ITEM_TYPES = frozenset(['File Geodatabase','CSV', 'Image', 'KML', 'Locator Package',
                                  'Map Document', 'Shapefile', 'Microsoft Word', 'PDF',
                                  'Microsoft Powerpoint', 'Microsoft Excel', 'Layer Package',
                                  'Mobile Map Package', 'Geoprocessing Package', 'Scene Package',
                                  'Tile Package', 'Vector Tile Package'])

ITEM_COPY_PROPERTIES = ['title', 'type', 'typeKeywords', 'description', 'tags',
                        'snippet', 'extent', 'spatialReference', 'name',
                        'accessInformation', 'licenseInfo', 'culture', 'url']

In [ ]:
ITEM_COPY_PROPERTIES = ['title', 'type', 'typeKeywords', 'description', 'tags',
                        'snippet', 'extent', 'spatialReference', 'name',
                        'accessInformation', 'licenseInfo', 'culture', 'url']

In [ ]:
def copy_item(source_item, dest_props):
    try:
        with tempfile.TemporaryDirectory() as temp_dir:
            item_properties = {}
            for property_name in ITEM_COPY_PROPERTIES:
                item_properties[property_name] = source_item[property_name]
            
            # overrides
            item_properties['title']= dest_props['title']
            
            if source_item.type == 'Web Map':
                item_properties['snippet']= dest_props['snippet']
                item_properties['tags']= dest_props['tags']
            
            data_file = None

            if source_item.type in TEXT_BASED_ITEM_TYPES:
                # If its a text-based item, then read the text and add it to the request.
                text = source_item.get_data(False)
                item_properties['text'] = text

            elif source_item.type in FILE_BASED_ITEM_TYPES:
                # download data and add to the request as a file
                data_file = source_item.download(temp_dir)

            thumbnail_file = source_item.download_thumbnail(temp_dir)
            metadata_file = source_item.download_metadata(temp_dir)

            # Add the item to the target portal, assign owner and folder
            target_item = target.content.add(item_properties, data_file, thumbnail_file,
                                             metadata_file, gis.users.me, folder_name)

            #Set sharing (privacy) information
            share_everyone = source_item.access == 'public'
            share_org = source_item.access in ['org', 'public']
            share_groups = []
            if source_item.access == 'shared':
                share_groups = source_item.groups

            target_item.share(share_everyone, share_org, share_groups)

            return target_item

    except Exception as copy_ex:
        print("\tError copying " + source_item.title)
        print("\t" + str(copy_ex))

In [ ]:
source_dashboard = gis.content.get('62f44e7da0864b1eb6f9d4bb5f9d2f54')
source_dashboard

In [ ]:
file_path = os.path.join('resources/clone-sdg-dashboards', 'sdgs-dashboard-metadata.json')
sdgs_meta = None
with open(file_path) as json_file:
    sdgs_meta = json.load(json_file)

In [ ]:
series_path = os.path.join('resources/clone-sdg-dashboards', 'series-code-info.json')
series_meta = None
with open(series_path) as s_json_file:
    series_meta = json.load(s_json_file)

In [ ]:
webmap_defaults = {
    "baseMap": {
        "baseMapLayers": [
            {
                "id": "VectorTile_6093",
                "type": "VectorTileLayer",
                "layerType": "VectorTileLayer",
                "title": "UN Light Gray Canvas Base",
                "styleUrl": "https://www.arcgis.com/sharing/rest/content/items/1b93d39e30e944479101cdec2351dca0/resources/styles/root.json",
                "itemId": "1b93d39e30e944479101cdec2351dca0",
                "visibility": True,
                "opacity": 1
            }
        ],
        "title": "UN Light Gray Basemap"
    },
    "extent": [
        [ -27.9236, -22.9613 ],
        [ 78.4241, 50.8733 ]
    ],
    "layerDefinition": {
        "definitionExpression": "latest_value IS NOT NULL"
    },
    "popupInfo": { "title": "<span style='font-size:18px'>{geoAreaName}</span>", "fieldInfos": [ { "fieldName": "series_release", "label": "Series Release", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox" }, { "fieldName": "series_code", "label": "Series Code", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox" }, { "fieldName": "series_description", "label": "Series Description", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox" }, { "fieldName": "geoAreaCode", "label": "Geographic Area Code", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox", "format": { "places": 0, "digitSeparator": True } }, { "fieldName": "X", "label": "X", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "Y", "label": "Y", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "ISO3CD", "label": "ISO3 Code", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox" }, { "fieldName": "geoAreaName", "label": "Geographic Area Name", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox" }, { "fieldName": "sliceId", "label": "Sliceid", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox", "format": { "places": 0, "digitSeparator": True } }, { "fieldName": "Units", "label": "Units", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox" }, { "fieldName": "Units_description", "label": "Units description", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox" }, { "fieldName": "F1990", "label": "1990", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox" }, { "fieldName": "F1991", "label": "1991", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F1992", "label": "1992", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F1993", "label": "1993", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F1994", "label": "1994", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F1995", "label": "1995", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F1996", "label": "1996", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F1997", "label": "1997", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F1998", "label": "1998", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F1999", "label": "1999", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2000", "label": "2000", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2001", "label": "2001", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2002", "label": "2002", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox" }, { "fieldName": "F2003", "label": "2003", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox" }, { "fieldName": "F2004", "label": "2004", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2005", "label": "2005", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2006", "label": "2006", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2007", "label": "2007", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2008", "label": "2008", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2009", "label": "2009", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2010", "label": "2010", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2011", "label": "2011", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2012", "label": "2012", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2013", "label": "2013", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2014", "label": "2014", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "F2015", "label": "2015", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox" }, { "fieldName": "F2016", "label": "2016", "isEditable": True, "tooltip": "", "visible": True, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "last_5_years_mean", "label": "Mean of the Last 5 Years", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox", "format": { "places": 2, "digitSeparator": True } }, { "fieldName": "latest_year", "label": "Latest Year", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox", "format": { "places": 0, "digitSeparator": False } }, { "fieldName": "latest_value", "label": "Latest Value", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox", "format": { "places": 0, "digitSeparator": True } }, { "fieldName": "latest_source", "label": "Latest Source", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox" }, { "fieldName": "latest_nature", "label": "Latest Nature", "isEditable": True, "tooltip": "", "visible": False, "stringFieldOption": "textbox" }, { "fieldName": "FID", "label": "FID", "isEditable": False, "tooltip": "", "visible": False, "stringFieldOption": "textbox" }, { "fieldName": "expression/expr0", "visible": True }, { "fieldName": "expression/expr1", "visible": True }, { "fieldName": "expression/expr2", "visible": True }, { "fieldName": "expression/expr3", "visible": True }, { "fieldName": "expression/expr4", "visible": True }, { "fieldName": "expression/expr5", "visible": True }, { "fieldName": "expression/expr6", "visible": True }, { "fieldName": "expression/expr7", "visible": True }, { "fieldName": "expression/expr8", "visible": True }, { "fieldName": "expression/expr9", "visible": True }, { "fieldName": "expression/expr10", "visible": True }, { "fieldName": "expression/expr11", "visible": True }, { "fieldName": "expression/expr12", "visible": True } ], "description": "For <b>{expression/expr11}</b>, the {series_description} was <b>{expression/expr12}</b>.", "showAttachments": True, "expressionInfos": [ { "name": "expr0", "title": "Custom", "expression": "// Write a script to return a value to show in the pop-up.\n// For example, get the average of 4 fields:\n// Average($feature.SalesQ1, $feature.SalesQ2, $feature.SalesQ3, $feature.SalesQ4)\n\nIIF(isEmpty($feature.F1995), \"\", Concatinate([1995, $feature.F1995],':'))", "returnType": "string" }, { "name": "expr1", "title": "Custom", "expression": "// Write a script to return a value to show in the pop-up.\n// For example, get the average of 4 fields:\n// Average($feature.SalesQ1, $feature.SalesQ2, $feature.SalesQ3, $feature.SalesQ4)\n\nIIF(isEmpty($feature.F2000), \"\", Concatenate([2000, $feature.F2000],':'))", "returnType": "string" }, { "name": "expr2", "title": "Custom", "expression": "// Write a script to return a value to show in the pop-up.\n// For example, get the average of 4 fields:\n// Average($feature.SalesQ1, $feature.SalesQ2, $feature.SalesQ3, $feature.SalesQ4)\n\nIIF(isEmpty($feature.F2005), \"\", Concatenate([2005, $feature.F2005],':'))", "returnType": "string" }, { "name": "expr3", "title": "Custom", "expression": "// Write a script to return a value to show in the pop-up.\n// For example, get the average of 4 fields:\n// Average($feature.SalesQ1, $feature.SalesQ2, $feature.SalesQ3, $feature.SalesQ4)\nIIF(isEmpty($feature.F2010), \"\", Concatenate([2010, $feature.F2010],':'))\n", "returnType": "string" }, { "name": "expr4", "title": "Custom", "expression": "// Write a script to return a value to show in the pop-up.\n// For example, get the average of 4 fields:\n// Average($feature.SalesQ1, $feature.SalesQ2, $feature.SalesQ3, $feature.SalesQ4)\n\nIIF(isEmpty($feature.F2012), \"\", Concatenate([2012, $feature.F2012],':'))\n", "returnType": "string" }, { "name": "expr5", "title": "Custom", "expression": "// Write a script to return a value to show in the pop-up.\n// For example, get the average of 4 fields:\n// Average($feature.SalesQ1, $feature.SalesQ2, $feature.SalesQ3, $feature.SalesQ4)\n\nIIF(isEmpty($feature.F2014), \"\", Concatenate([2014, $feature.F2014],':'))\n", "returnType": "string" }, { "name": "expr6", "title": "Custom", "expression": "// Write a script to return a value to show in the pop-up.\n// For example, get the average of 4 fields:\n// Average($feature.SalesQ1, $feature.SalesQ2, $feature.SalesQ3, $feature.SalesQ4)\n\nIIF(isEmpty($feature.F2015), \"\", Concatenate([2015, $feature.F2015],':'))\n", "returnType": "string" }, { "name": "expr7", "title": "Custom", "expression": "// Write a script to return a value to show in the pop-up.\n// For example, get the average of 4 fields:\n// Average($feature.SalesQ1, $feature.SalesQ2, $feature.SalesQ3, $feature.SalesQ4)\n\nIIF(isEmpty($feature.F2016), \"\", Concatenate([2016, $feature.F2016],':'))\n", "returnType": "string" }, { "name": "expr8", "title": "Custom", "expression": "// Write a script to return a value to show in the pop-up.\n// For example, get the average of 4 fields:\n// Average($feature.SalesQ1, $feature.SalesQ2, $feature.SalesQ3, $feature.SalesQ4)\n\nIIF(isEmpty($feature.Latest_Value), \"\", Concatenate(['Latest Year (Value) - ',$feature.Latest_Year, '(',$feature.Latest_Value,'%)'],' '))\n", "returnType": "string" }, { "name": "expr9", "title": "Custom", "expression": "// Write a script to return a value to show in the pop-up.\n// For example, get the average of 4 fields:\n// Average($feature.SalesQ1, $feature.SalesQ2, $feature.SalesQ3, $feature.SalesQ4)\n\nIIF(isEmpty($feature.F2011), \"\", Concatenate([2011, $feature.F2011],':'))\n", "returnType": "string" }, { "name": "expr10", "title": "Custom", "expression": "// Write a script to return a value to show in the pop-up.\n// For example, get the average of 4 fields:\n// Average($feature.SalesQ1, $feature.SalesQ2, $feature.SalesQ3, $feature.SalesQ4)\n\nIIF(isEmpty($feature.F2013), \"\", Concatenate([2013, $feature.F2013],':'))\n", "returnType": "string" }, { "name": "expr11", "title": "Latest Year", "expression": "var latest_year = $feature[\"latest_year\"];\nreturn Text(latest_year);", "returnType": "string" }, { "name": "expr12", "title": "Latest Value", "expression": "var latest_value = $feature[\"latest_value\"];\nif ($feature.Units == 'PERCENT') {\n    latest_value = latest_value + ' %';\n}\nreturn latest_value;", "returnType": "string" } ], "mediaInfos": []}
}

In [ ]:
dashboards_to_create = [
    {
        "goal_id": "2",
        "series_code": "C020101",
        "indicator_name": "Prevalence of undernourishment (%)",
        "hub_item_id": '8c0866ad73ae486ba647f5490fed5529'
    }
]

In [ ]:
def get_widget_by_type(json_def, widget_type):
    if widget_type == 'headerPanel':
        return json_def['headerPanel']
    
    return [item for item in json_def['widgets'] if item['type'] == widget_type][0]    

In [ ]:
output_log_rows = []
for i, dest_props in enumerate(series_meta):    
    
    # testing break after 5 created
    if i == 3:
        break
        
    print ('STARTING CLONE FOR {} - {}'.format(dest_props['series_code'], dest_props['indicator_name']))

    # get sdg meta data info
    meta_data = sdgs_meta[(dest_props['goal_code'])]
    
#     print ('getting feature service ...')
    # get the new feature service
    feature_service = target.content.get(dest_props['hub_item_id'])
    fs_url = feature_service['url']
    fs_id = fs_url[fs_url.index('services/')+9:fs_url.index('/FeatureServer')]
    
    fs_title = feature_service['title']
    splits = fs_title.split(':')[0].split(' ')[1].split('.')
    sdg_goal = splits[0]
    sdg_target = '{}.{}'.format(splits[0], splits[1])
    sdg_indicator = '{}.{}.{}'.format(splits[0], splits[1], splits[2])
    
    fl = FeatureLayer.fromitem(feature_service)
    
    # basic new props for copied dashboard
    new_dash_props = {
        'title': fs_title
    }
    
#     print ('copying source dashboard ...')
    # copy the dashboard
    copied_dashboard = copy_item(source_dashboard, new_dash_props)
    
    # get the dashboard's JSON data
    cd_json = copied_dashboard.get_data(True)
    
    # get the map widget to get the webmap id
    map_widget = get_widget_by_type(cd_json, 'mapWidget')
    map_widget_item_id = map_widget['itemId']
    source_wm = target.content.get(map_widget_item_id)
    
    # create new webmap
#     print ('creating new webmap ...')
    new_wm_props = {
        'title': feature_service['title'],
        'snippet': feature_service['snippet'],
        'tags': feature_service['tags']
    }
    new_wm = WebMap()
    new_wm.add_layer(feature_service)
    new_wm_item = new_wm.save(new_wm_props)
    
#     print ('updating and saving new webmap ...')
    new_wm_json = new_wm_item.get_data(True)
    new_wm_json['operationalLayers'][0]['id'] = fs_id
    new_wm_json['operationalLayers'][0]['popupInfo'] = webmap_defaults['popupInfo']
    new_wm_json['operationalLayers'][0]['layerDefinition'] = webmap_defaults['layerDefinition']
    new_wm_json['baseMap'].update(webmap_defaults['baseMap'])
    
    updated_wm_props = {
        'tags': ['country profile', 'series web map'], 
        'extent': webmap_defaults['extent'],
        'categories': ["/Categories/Goal {}/Target {}".format(sdg_goal, sdg_target)]
    }
    new_wm_item.update(updated_wm_props, data=json.dumps(new_wm_json))
    
    # share to public
    new_wm_item.share(everyone=True)
    
    # move to working folder (optional)
    if folder_name is not None:
        new_wm_item.move(folder_name)
    
    #### --- Update the Dashboard widgets
    print ('BEGIN -- Dashboard updates')

    #### update the header
#     print ('updating header ...')
    header_json = get_widget_by_type(cd_json, 'headerPanel')
    header_json.update(meta_data['headerPanel'])
    
    #### update map widget    
#     print ('updating map widget ...')
    map_widget['name'] = new_dash_props['title']
    map_widget['itemId'] = new_wm_item.id
    map_widget['layers'][0]['layerId'] = fs_id
    
    #### update list widget 
#     print ('updating list widget ...')
    # update the datasource id
    list_widget = get_widget_by_type(cd_json, 'listWidget')
    old_id = list_widget['datasets'][0]['dataSource']['id'].split('#')
    new_id = '{}#{}'.format(old_id[0], fs_id)
    list_widget['datasets'][0]['dataSource']['id'] = new_id
    
    # update text
    list_widget['text'] = meta_data['listWidget']['text']
    
    # update the caption
    soup = BeautifulSoup(list_widget['caption'])
    span_el = soup.find_all('span')[1]
    span_el['style'] = 'color:{}'.format(meta_data['listWidget']['captionColor'])
        
    new_caption_text = dest_props['indicator_name']
    span_el.string = new_caption_text
    
    list_widget['caption'] = str(soup.p)
    
    #### update indicator widget 
#     print ('updating indicator widget ...')
    ind_widget = get_widget_by_type(cd_json, 'indicatorWidget')
    ind_widget['defaultSettings'].update(meta_data['indicatorWidget']['defaultSettings'])
    ind_widget['datasets'][0]['dataSource']['id'] = new_id
    
    #### update serial chart widget 
#     print ('updating serial chart widget ...')
    sc_widget = get_widget_by_type(cd_json, 'serialChartWidget')
    
    # update the graph color
    sc_widget['graphs'][0].update(meta_data['serialChartWidget']['graphs'])
    
    # update the datasource id
    sc_widget['datasets'][0]['dataSource']['id'] = new_id
        
    #### update embedded content widget
#     print ('updating embedded content widget ...')
    em_widget = get_widget_by_type(cd_json, 'embeddedContentWidget')
    em_widget.update(meta_data['embeddedContentWidget'])
    
    #### update details widget
#     print ('updating details widget ...')
    dt_widget = get_widget_by_type(cd_json, 'detailsWidget')
    dt_widget['datasets'][0]['dataSource']['id'] = new_id
    
    #### ****************    
    #### commit changes to new dashboard
    print ('updating new dashboard ..')
    copied_dashboard.update({
        'tags': 'country profile, series dashboard',
        'categories': ["/Categories/Goal {}/Target {}".format(sdg_goal, sdg_target)]
    }, data=json.dumps(cd_json))
    
    # share to public
    copied_dashboard.share(everyone=True)
    
    output_log_rows.append([sdg_goal, sdg_target, sdg_indicator, dest_props['series_code'], new_wm_item.id, copied_dashboard.id])
    
    print ('END   -- Dashboard updates')
    print ('*************************')
    
print ('done.')

In [ ]:
import csv
with open('outputlog.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['goal', 'target','indicator', 'series', 'webmap id', 'dashboard id'])
    writer.writerows(output_log_rows)